Subtract convolved single lens magnification map (analytic) from this and then plot fractional deviations:
- Want to explore how small of a mass ratio you can detect the second planet
- Plot difference of this magnification map and most massive planet and star
- See how small you can make the small planet before it is nondetectable
- Later: add additional planets

(Triple - binary) / binary
- Plot grid of changing seperation and planet mass ratio to see how deviation changes
- Later on explore multiple planets and then adding moons...
- Later later on add in orbital motion and see what happens

Trajectories:
- Through fractional deviation map

Importing relevant libraries and timing custom library imports

In [ ]:
%matplotlib tk

%load_ext autoreload
%autoreload 2

import time as t

import numpy as np
import matplotlib.pyplot as plt

import matplotlib.patches as patches
import matplotlib.collections as collections

import sys
sys.path.append('..')

# Timing imports
start = t.time()
from IRSMicroLensing import IRSCaustics as IRSC
print(f"Import time: {t.time() - start:.2f} seconds")

start = t.time()
from IRSMicroLensing import IRSFunctions as IRSF
print(f"Import time: {t.time() - start:.2f} seconds")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Import time: 0.00 seconds
Import time: 0.00 seconds


Defining binary lens attributes and finding the most optimal angular width and thickness to find all central caustics

In [5]:
# Big planet position
s1 = 0.8
alpha1 = 0

# Small planet position
s2 = s1
alpha2 = 30

# Small planet mass ratios
# upper_bound = -7
# lower_bound = -16

upper_bound = -3
lower_bound = -12

# Colormap
cmap = 'gray'

# Defining binary lens parameters
binary_lens_attributes = [
    [0, 0, 0.01, 1],
    [s1*np.cos(np.deg2rad(alpha1)), s1*np.sin(np.deg2rad(alpha1)), 0.01, 1e-3]
]

# Creating a list of all lenses
complete_lens_attributes = binary_lens_attributes.copy()

for i in range(upper_bound, lower_bound, -1):
    complete_lens_attributes.append(
        [s2*np.cos(np.deg2rad(alpha2)), s2*np.sin(np.deg2rad(alpha2)), 0.01, 10**(i)] # Small mass
    )

# Finding best angular width and thickness
ang_width, thickness, caustic_cusps = IRSC.IRSCaustics.ang_width_thickness_calculator(lens_att=complete_lens_attributes)

print(f"Angular width: {ang_width}")
print(f"Thickness: {thickness}")

Angular width: 0.07680319680319682
Thickness: 0.07680319680319682


Creating a list of lens attributes that includes each trinary configuration

In [6]:
# Probing mass space to see what the smallest mass detectable is
trinary_lens_attributes_list = []

for i in range(upper_bound, lower_bound, -1):
    trinary = binary_lens_attributes.copy()
    trinary.append(
        [0.8, 0.001, 0.01, 10**(i)] # Small mass
    )

    trinary_lens_attributes_list.append(trinary)

print(f'Number of trinary systems: {len(trinary_lens_attributes_list)}')

Number of trinary systems: 9


Initializing magnification map calculator objects

In [22]:
binary_param_dict = {'pixels': 5000, 'ang_width': ang_width, 'lens_att': binary_lens_attributes, 'thickness': thickness, 'num_r': 12000, 'num_theta': 3000}
binary_mag_map = IRSC.IRSCaustics(annulus_param_dict=binary_param_dict)

binary_magnifications = binary_mag_map.plot(show_lenses=False, cmap=cmap, show_plot=True)

plt.show()

Creating mesh grid: 0.074 seconds
Calculating source pixels: 0.365 seconds
Calculating indices of translated pixel after deflection: 0.327 seconds
Calculating translated pixels: 0.513 seconds
Finding pixel repetitions and counts: 0.212 seconds
Incrementing pixel magnifications based on counts and repetitions: 0.224 seconds
Plotting magnification map: 0.234 seconds
---------------------
Total time: 2.292 seconds


Defining source profile for convolution

In [8]:
radius = 1e-3
LD = 0.5

source_profile = IRSF.IRSFunctions.source_profile(ang_res=binary_mag_map.param_dict['ang_res'], rad=radius, profile_type='LD', LD=LD)

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot()
img = ax.imshow(source_profile, cmap='afmhot', extent=[-radius, radius, -radius, radius], aspect='auto', interpolation='nearest')
plt.colorbar(img, ax=ax, label='Source Brightness')

ax.set_xlabel('X [$\\theta_E$]')
ax.set_ylabel('Y [$\\theta_E$]')
ax.set_title('Source Profile')

ax.set_aspect('equal')

plt.show()

Convolving binary brightnesses with source profile

In [9]:
convolved_binary_brightnesses = binary_mag_map.convolve(source_profile=source_profile)

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot()
img = ax.imshow(convolved_binary_brightnesses, cmap=cmap, extent=[-binary_mag_map.ang_width/2, binary_mag_map.ang_width/2, -binary_mag_map.ang_width/2, binary_mag_map.ang_width/2])
plt.colorbar(img, ax=ax, label='Convolved Brightness')

ax.set_title('Convolved Brightnesses of Binary Lens')
ax.set_xlabel('X [$\\theta_E$]')
ax.set_ylabel('Y [$\\theta_E]$')

plt.show()

Convolving source profile with magnification map: 0.871 seconds


Calculating convolved brightnesses and their deviation from binary brightness for each trinary configuration

In [10]:
# Defining list of instances of IRSCaustics for trinary lens
trinary_mag_map_list = []

# Creating a list of convolved brightnesses for each trinary lens
convolved_trinary_brightnesses_list = []

# List of convolved brightness difference for each trinary lens
convolved_brightness_differences_list = []

# Main for loop for each trinary lens configuration
for i in range(len(trinary_lens_attributes_list)):
    # Creating instance of IRSCaustics for trinary lens
    trinary_param_dict = binary_param_dict.copy()
    trinary_param_dict['lens_att'] = trinary_lens_attributes_list[i]
    trinary_mag_map_list.append(IRSC.IRSCaustics(annulus_param_dict=trinary_param_dict))

    print()
    print(f"Calculating trinary lens magnification map {i+1}/{len(trinary_lens_attributes_list)}")
    trinary_magnifications = trinary_mag_map_list[i].plot(show_lenses=False, cmap=cmap, show_plot=False)

    # Convolving each magnification map with the source profile
    convolved_trinary_brightnesses_list.append(trinary_mag_map_list[i].convolve(source_profile=source_profile))

    # Plotting difference between convolved magnification maps
    convolved_brightness_differences_list.append((convolved_trinary_brightnesses_list[i] - convolved_binary_brightnesses) / convolved_binary_brightnesses)


Calculating trinary lens magnification map 1/9
Creating mesh grid: 0.082 seconds
Calculating source pixels: 1.176 seconds
Calculating indices of translated pixel after deflection: 0.425 seconds
Calculating translated pixels: 0.693 seconds
Finding pixel repetitions and counts: 0.328 seconds
Incrementing pixel magnifications based on counts and repetitions: 0.234 seconds
---------------------
Total time: 3.236 seconds
Convolving source profile with magnification map: 0.797 seconds

Calculating trinary lens magnification map 2/9
Creating mesh grid: 0.075 seconds
Calculating source pixels: 1.405 seconds
Calculating indices of translated pixel after deflection: 0.343 seconds
Calculating translated pixels: 0.774 seconds
Finding pixel repetitions and counts: 0.281 seconds
Incrementing pixel magnifications based on counts and repetitions: 0.297 seconds
---------------------
Total time: 3.478 seconds
Convolving source profile with magnification map: 0.797 seconds

Calculating trinary lens magn

Finding maximum and minimum deviation for all trinary configurations

In [11]:
# Finding max of all convolved brightness differences
for i in range(len(convolved_brightness_differences_list)):
    print(np.max(convolved_brightness_differences_list[i]))

absolute_max = np.max(convolved_brightness_differences_list)
print(f"\nAbsolute max: {absolute_max}")

print()
# Finding max of all convolved brightness differences
for i in range(len(convolved_brightness_differences_list)):
    print(np.min(convolved_brightness_differences_list[i]))

absolute_min = np.min(convolved_brightness_differences_list)
print(f"\nAbsolute min: {absolute_min}")

2.186792617191394
0.4641218900937613
0.0580457441062578
0.015022898080392463
0.014345099542052252
0.014159692291684046
0.014159692291690586
0.014159692291685328
0.014159692291693152

Absolute max: 2.186792617191394

-0.7746292208374551
-0.4340654386522729
-0.08918589664577507
-0.01466061924495023
-0.013666407171089376
-0.013770782402839393
-0.013770782402836898
-0.013770782402838145
-0.013770782402834279

Absolute min: -0.7746292208374551


Plotting the deviation for each trinary configuration

In [12]:
fig = plt.figure(figsize=(12, 12))
axes = fig.subplots(3, 3)

fig.suptitle('\nConvolved Brightness Fractional Differences for Trinary Lenses', fontsize=16)

for i in range(len(trinary_lens_attributes_list)):
    if i < 3:
        ax = axes[0, i]
    elif i < 6:
        ax = axes[1, i-3]
    else:
        ax = axes[2, i-6]

    img = ax.imshow(convolved_brightness_differences_list[i], cmap=cmap, extent=[-binary_mag_map.ang_width/2, binary_mag_map.ang_width/2, -binary_mag_map.ang_width/2, binary_mag_map.ang_width/2], vmin=absolute_min, vmax=absolute_max)
    
    ax.set_title(f"Planet Mass Ratio: {trinary_lens_attributes_list[i][2][3] / binary_lens_attributes[1][3]:.2e}")
    # ax.set_xlabel('X [$\\theta_E$]')
    # ax.set_ylabel('Y [$\\theta_E]$')
    ax.axis('off')
    ax.set_xlim(-binary_mag_map.ang_width/2, binary_mag_map.ang_width/2)
    ax.set_ylim(-binary_mag_map.ang_width/2, binary_mag_map.ang_width/2)

    props = dict(boxstyle='round', facecolor='white', edgecolor='lightgray', alpha=0.8)
    ax.text(0.02, 0.98, f'Max = {round(np.max(convolved_brightness_differences_list[i]), 18)}\nMin = {round(np.min(convolved_brightness_differences_list[i]), 18)}', va='top', zorder=10, bbox=props, transform=ax.transAxes)

cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
cbar_ax.axis('off')

bar = fig.colorbar(img, ax=cbar_ax, location='right', fraction=1.5, aspect=20)
bar.set_label('Fractional Difference')

fig.savefig('Trinary Lens Convolved Convolved Fractional Difference.png', dpi=500)

plt.show()

Defining source trajectory

In [25]:
# Removing any present lines from magnification map
for child in binary_mag_map.ax_c.get_children():
    if isinstance(child, (collections.PathCollection, collections.PatchCollection)):
        child.remove()

# Impact parameter
u = -2e-3

# Angle from x axis
alpha = 45

# Number of points
num_points = 100

# Discretizing the source trajectory
xs_prime = np.linspace(-binary_mag_map.ang_width/np.sqrt(2), binary_mag_map.ang_width/np.sqrt(2), num_points).reshape(1, num_points)
ys_prime = np.ones((1, num_points)) * u

points_prime = np.vstack((xs_prime, ys_prime))

# Rotating the source trajectory
rotation_matrix = np.array([[np.cos(np.radians(alpha)), -np.sin(np.radians(alpha))],
                             [np.sin(np.radians(alpha)), np.cos(np.radians(alpha))]])

points = np.dot(rotation_matrix, points_prime)

# Extracting x and y coordinates
xs = points[0]
ys = points[1]

# Plotting the source trajectory
circles = [patches.Circle((x, y), radius=radius, color='red', alpha=0.5, fill=False) for x, y in zip(xs, ys)]
c = collections.PatchCollection(circles, match_original=True, zorder=50)
binary_mag_map.ax_c.add_collection(c)
# binary_mag_map.ax_c.scatter(xs, ys, s=radius, color='red', zorder=50)

binary_mag_map.fig_c.show()

In [61]:
import numpy as np

def compute_fractional_circle_sum_centered(brightness, x_center, y_center, radius, ax, supersample=11):
    """
    Compute the brightness sum within a circle of given radius, centered at (x_center, y_center),
    using a coordinate system where (0, 0) is the center of the array.
    
    Parameters:
        brightness (2D array): The brightness grid, shape (H, W)
        x_center, y_center (float): Circle center in (0,0)-centered coordinates
        radius (float): Radius of the circle in pixels
        supersample (int): Supersampling resolution per pixel
        
    Returns:
        float: Weighted sum of brightness values within the circle
    """
    H, W = brightness.shape
    cy, cx = H // 2, W // 2  # center pixel coordinates

    print(cy, cx)

    # Convert from centered coordinates to array indices
    j_center = x_center + cx
    i_center = y_center + cy

    floor_x = int(np.floor(j_center - radius))
    ceil_x = int(np.ceil(j_center + radius))
    floor_y = int(np.floor(i_center - radius))
    ceil_y = int(np.ceil(i_center + radius))

    print(floor_x, ceil_x)
    print(floor_y, ceil_y)

    total = 0.0

    # Supersampling grid
    ss = supersample
    offsets = np.linspace(-0.5 + 0.5/ss, 0.5 - 0.5/ss, ss)
    sub_x, sub_y = np.meshgrid(offsets, offsets)

    for i in range(floor_y, ceil_y + 1):
        for j in range(floor_x, ceil_x + 1):
            if i < 0 or i >= H or j < 0 or j >= W:
                continue
            
            # Get physical coordinates of the pixel center
            x = j - cx
            y = i - cy

            print(x, y)

            dx = x + sub_x - x_center
            dy = y + sub_y - y_center

            ax.scatter(dx+x_center, dy+y_center, s=1, color='blue', alpha=0.5)

            distances_squared = dx**2 + dy**2
            fraction = np.sum(distances_squared <= radius**2) / (ss * ss)

            total += brightness[i, j] * fraction

    return total


In [63]:
# Random 2D brightness grid
brightness = np.random.randint(0, 10, (101, 101))

# Circle parameters
x_center = 0
y_center = 0
radius = 2

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot()

ax.imshow(brightness, extent=[-50.5, 50.5, -50.5, 50.5])

# Compute brightness sum
result = compute_fractional_circle_sum_centered(brightness, x_center, y_center, radius, ax)
print(result)

circle = patches.Circle((x_center, y_center), radius=radius, edgecolor='red', fill=True, alpha=0.5, facecolor='gray')
ax.add_patch(circle)

plt.show()

50 50
48 52
48 52
-2 -2
-1 -2
0 -2
1 -2
2 -2
-2 -1
-1 -1
0 -1
1 -1
2 -1
-2 0
-1 0
0 0
1 0
2 0
-2 1
-1 1
0 1
1 1
2 1
-2 2
-1 2
0 2
1 2
2 2
49.65289256198347
